In [5]:
import pandas as pd
rt = pd.read_excel("gpt_evaluation_RT.xlsx")

In [6]:
print(rt.head())

  Unnamed: 0 Unnamed: 1                                         Unnamed: 2  \
0      Index         ID                                           Abstract   
1          1   CHE-1045  The case involves a labor contract dispute bet...   
2          2   CHE-1044  Abstract\n\nArt. 154 ff. Swiss Private Interna...   
3          3   CHE-1333  Art. 15, 117, and 122 of the Swiss Private Int...   
4          4   CHE-1050  Abstract\n\nInternational Private Law. The abi...   

  Does it contain all the information relevant for this category and does it use the correct language to describe all important aspects? (when it is not ctrl c-v)  \
0                                  Abstract Accuracy                                                                                                                 
1                                           40 (2.3)                                                                                                                 
2                                